In [1]:
import yfinance as yf

# BTC-USD için tarihsel verileri çekme
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')


[*********************100%%**********************]  1 of 1 completed


In [2]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Sadece kapanış fiyatını kullanacağız
close_prices = data[['Close']].values

# Veriyi ölçeklendirme
scaler = MinMaxScaler(feature_range=(0,1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Zaman serisi verisini eğitim setine dönüştürme
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 100
X, y = create_dataset(close_prices_scaled, time_step)
X = X.reshape(X.shape[0],X.shape[1] , 1) # LSTM için gerekli şekle getirme


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.layers import Attention

# Model oluşturma
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(100, 1)),
    Dropout(0.2),
    # Dikkat katmanı
    Attention(),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')


2024-02-03 21:13:20.532996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Exception encountered when calling layer "attention" (type Attention).

Attention layer must be called on a list of inputs, namely [query, value] or [query, value, key]. Received: Tensor("Placeholder:0", shape=(None, 100, 50), dtype=float32).

Call arguments received by layer "attention" (type Attention):
  • inputs=tf.Tensor(shape=(None, 100, 50), dtype=float32)
  • mask=None
  • training=None
  • return_attention_scores=False
  • use_causal_mask=False

In [ ]:
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


In [ ]:
# Tahminleri ölçeklendirilmiş değerlerden gerçek değerlere çevirme
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)

# Gerçek değerler ile tahminleri karşılaştırma
